In [1]:
import plotly.express as px
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import altair as alt

#import csv
pd.set_option('display.max_rows', None)  # Display all rows

In [14]:
day = "11"
month = "10"
dataset = "ble"

base = -1
base_person = -1

date = day + "_" + month #change date accordingly
date_year = "2023-" + month + "-" + day
if date == "11_10":
    base = 1
elif date == "13_10":
    base = 2
elif date == "24_08":
    base = 0
    base_person = 1
else:
    base = 0

if base == 1 or base ==2:
    folder_path = dataset + '_base/'
elif base_person:
    folder_path = dataset + '_base_person/'
else:
    folder_path = dataset +'_'+ date + '/'

def process_file(file_number, date):
    file_path = os.path.join(folder_path, f'{dataset}_{date}_{file_number}.csv')

    df = pd.read_csv(file_path, parse_dates=['ts'])

    # Remove first empty column if it exists
    if df.shape[1] == 4:

        df = df.drop(df.columns[0], axis=1)
    # Remove entries with 'gabarit' equal to 1 or 2
    df_filt = df[(df['id'] == 2) | (df['id'] == 1)]

    # Write the updated DataFrame back to the CSV file
    df.to_csv(file_path, index = False)
    # Reset the index of the DataFrame
    df_filt = df_filt.reset_index(drop = True)

    return df_filt

df1 = process_file(2,date)
df2 = process_file(1,date)
df3 = process_file(3,date)


In [15]:
def plot_ble(df,station):
    # Plotly line graph
    color_discrete= {'1': 'tomato', '2': 'conrflowerblue'}

    fig = px.line(df, x='ts', y='rssi', color='id', color_discrete_map=color_discrete,
                  title=f'Line Graph for Station {station}')
    fig.update_layout(xaxis_title='Timestamp', yaxis_title='RSSI', legend_title='ID')
    fig.show()
plot_ble(df1,station =1)
plot_ble(df2, station =2)
plot_ble(df3, station = 3)



c:\Users\isacc\miniconda3\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



c:\Users\isacc\miniconda3\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



c:\Users\isacc\miniconda3\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [16]:
# FIND BASE VALUES
def find_ref_ble_1_2(df1,df2,df3, start_time, end_time):

    ref_values = np.zeros((3,3))
    #############################################################
    #                              station 1 station 2 station 3
    #beacon placed in station 1     X         X         X
    #beacon placed in station 2     X         X         X
    #beacon placed in station 3     X         X         X

    #beacon 2 placed in beggining (1)
    #beacon 1 placed in end on production line (2)
    for beacon in [1,2]:
        df1_id = df1[df1['id'] == beacon]
        df2_id = df2[df2['id'] == beacon]
        df3_id = df3[df3['id'] == beacon]

        

        # Filter DataFrame based on the time range
        df1_1 = df1_id[(df1_id['ts'] >= start_time) & (df1_id['ts'] <= end_time)]
        #av_{station}_{time_interval}
        med1_1 = df1_1['rssi'].median()
        av1_1 = df1_1['rssi'].mean()

        df2_1 = df2_id[(df2_id['ts'] >= start_time) & (df2_id['ts'] <= end_time)]
        med2_1 = df2_1['rssi'].median()
        av2_1 = df2_1['rssi'].mean()

        df3_1 = df3_id[(df3_id['ts'] >= start_time) & (df3_id['ts'] <= end_time)]
        med3_1 = df3_1['rssi'].median()
        av_3_1 = df3_1['rssi'].mean()


        ref_values[beacon-1,:] = [av1_1,av2_1,av_3_1]      


    return ref_values

def find_ref_3(df1,df2,df3, ref_values,start_time, end_time):
    beacon = 1 #placed in station 3
    df1_id = df1[df1['id'] == beacon]
    df2_id = df2[df2['id'] == beacon]
    df3_id = df3[df3['id'] == beacon]
    # Filter DataFrame based on the time range
    df1_1 = df1_id[(df1_id['ts'] >= start_time) & (df1_id['ts'] <= end_time)]
    med1_1 = df1_1['rssi'].median()
    av1_1 = df1_1['rssi'].mean()

    df2_1 = df2_id[(df2_id['ts'] >= start_time) & (df2_id['ts'] <= end_time)]
    med2_1 = df2_1['rssi'].median()
    av2_1 = df2_1['rssi'].mean()

    df3_1 = df3_id[(df3_id['ts'] >= start_time) & (df3_id['ts'] <= end_time)]
    med3_1 = df3_1['rssi'].median()
    av_3_1 = df3_1['rssi'].mean()

    ref_values[2,:] = [av1_1,av2_1,av_3_1]
    #ref_values[2,:] = [-60, -70, -50]

    return ref_values

if base == 1:
    start_time = '2023-10-11 19:00:00'
    end_time = '2023-10-12 07:00:00'
    ref_values = find_ref_ble_1_2(df1,df2,df3, start_time, end_time)

    #start_time = '2023-10-13 19:00:00'
    #end_time = '2023-10-14 07:00:00'
    #ref_values = find_ref_3(df1,df2,df3, ref_values,start_time,end_time)
    print(ref_values)



if base == 2:
    start_time = '2023-10-13 19:00:00'
    end_time = '2023-10-14 07:00:00'
    ref_values = find_ref_3(df1,df2,df3, ref_values,start_time,end_time)
    print(ref_values)




[[-80.         -44.23451777 -66.21052632]
 [-66.46572438 -85.8369906  -87.02805611]
 [  0.           0.           0.        ]]


In [17]:


#                              station 1 station 2 station 3
#beacon placed in station 1(beggining)     X         X         X
#beacon placed in station 2(end)           X         X         X
#beacon placed in station 3(elevator)      X         X         X
ref_values = np.array([[-66.46572438, -85.8369906,  -87.02805611],
                    [-80.0,         -44.23451777, -66.21052632],
                    [-68.76865672, -77.92592593, -80.08108108]])

In [26]:
def analyse_df(df1, df2, df3, beacon):
    # Filter DataFrames for the given id value
    df1_id = df1[(df1['id'] == beacon) & (df1['rssi'] != -100)].reset_index(drop=True)
    df2_id = df2[(df2['id'] == beacon) & (df2['rssi'] != -100)].reset_index(drop=True)
    df3_id = df3[(df3['id'] == beacon) & (df3['rssi'] != -100)].reset_index(drop=True)

    id_max = df1_id.shape[0]
    df1_id = df1_id.loc[:id_max]
    df2_id = df2_id.loc[:id_max]
    df3_id = df3_id.loc[:id_max]

    df_rssi = pd.DataFrame(columns=['ts', 'rssi1', 'rssi2', 'rssi3', 'station'])
    id = 0

    # Iterate through timestamps in df1
    for index, row in df1_id.iterrows():
        ts_current = pd.to_datetime(row['ts'])
        df_rssi.loc[id, 'ts'] = ts_current
        ts_end = ts_current + pd.Timedelta(seconds=30)

        # Initialize variables for RSSI values and sum counters
        sum1 = 0
        sum2 = 0
        sum3 = 0
        rssi_df1 = 0
        rssi_df2 = 0
        rssi_df3 = 0

        # Calculate average RSSI values for df1
        rssi_curr_df1 = row['rssi']
        rssi_df1 = rssi_curr_df1
        sum1 += 1
        if index != 0:
            rssi_prev_df1 = df1_id.loc[index - 1, 'rssi']
            rssi_df1 = rssi_df1 + rssi_prev_df1
            sum1 += 1
        if index <= id_max - 2:
            rssi_next_df1 = df1_id.loc[index + 1, 'rssi']
            rssi_df1 = rssi_df1 + rssi_next_df1
            sum1 += 1
        rssi_df1 = rssi_df1 / sum1
        df_rssi.loc[id, 'rssi1'] = rssi_df1

        # Find the corresponding NEXT timestamp in df2 and df3
        df2_subset_next = df2_id[(df2_id['ts'] >= ts_current) & (df2_id['ts'] <= ts_end)]
        df3_subset_next = df3_id[(df3_id['ts'] >= ts_current) & (df3_id['ts'] <= ts_end)]

        if not df2_subset_next.empty:
            # Find the first occurrence of the next timestamp in df2 and df3
            ts_next_df2 = df2_subset_next.iloc[0]['ts']
            rssi_next_df2 = df2_subset_next.iloc[0]['rssi']
            rssi_df2 = rssi_next_df2
            sum2 += 1
        if not df3_subset_next.empty:
            ts_next_df3 = df3_subset_next.iloc[0]['ts']
            rssi_next_df3 = df3_subset_next.iloc[0]['rssi']
            rssi_df3 = rssi_next_df3
            sum3 += 1



        # Find the corresponding PREVIOUS timestamp in df2 and df3
        df2_subset_prev = df2_id[(df2_id['ts'] <= ts_current)].tail(1)
        df3_subset_prev = df3_id[(df3_id['ts'] <= ts_current)].tail(1)

        # Calculate average RSSI values for df2 and df3
        if not df2_subset_prev.empty:
            ts_prev_df2 = df2_subset_prev.iloc[0]['ts']
            rssi_prev_df2 = df2_subset_prev.iloc[0]['rssi']
            rssi_df2 += rssi_prev_df2
            sum2 += 1
        if not df3_subset_prev.empty:
            ts_prev_df3 = df3_subset_prev.iloc[0]['ts']
            rssi_prev_df3 = df3_subset_prev.iloc[0]['rssi']
            rssi_df3 += rssi_prev_df3
            sum3 += 1

        # Calculate final average RSSI values for df2 and df3
        if sum2 != 0:
            rssi_df2 = rssi_df2 / sum2
            df_rssi.loc[id, 'rssi2'] = rssi_df2
        else:
            df_rssi.loc[id, 'rssi2'] = -100

        if sum3 != 0:
            rssi_df3 = rssi_df3 / sum3
            df_rssi.loc[id, 'rssi3'] = rssi_df3
        else:
            df_rssi.loc[id, 'rssi3'] = -100
            
        id += 1
    return df_rssi




df_rssi1 = analyse_df(df1,df2,df3, beacon = 1) #beacon 1

df_rssi2 = analyse_df(df1,df2,df3, beacon = 2) #beacon 2
print(df_rssi1)


                    ts rssi1 rssi2 rssi3 station
0  2023-10-11 19:52:58 -60.0 -40.0 -75.0     NaN
1  2023-10-11 20:47:57 -60.0 -40.0 -75.0     NaN


In [19]:
def state_df_person(df):
    #1 fim 18:02:00 -> 18:06:00
    #2 inicio 17:57:30 -> 18:01:00
    #3 monta cargas 18:07:00 -> 18:10:00
    df['real station'] = 0

    # Define the time frame
    start_time = pd.to_datetime('2023-08-24 18:02:00')
    end_time = pd.to_datetime('2023-08-24 18:05:20')

    # Set 'real station' to 1 for rows within the specified time frame
    df.loc[(df['ts'] >= start_time) & (df['ts'] <= end_time), 'real station'] = 2

    # Define the time frame
    start_time = pd.to_datetime('2023-08-24 17:57:35')
    end_time = pd.to_datetime('2023-08-24 18:00:40')

    # Set 'real station' to 2 for rows within the specified time frame
    df.loc[(df['ts'] >= start_time) & (df['ts'] <= end_time), 'real station'] = 1

    # Define the time frame
    start_time = pd.to_datetime('2023-08-24 18:07:00')
    end_time = pd.to_datetime('2023-08-24 18:10:00')

    # Set 'real station' to 3 for rows within the specified time frame
    df.loc[(df['ts'] >= start_time) & (df['ts'] <= end_time), 'real station'] = 3

    return df


if date == "24_08":
    df_person = state_df_person(df_rssi2)

print(ref_values)


[[-66.46572438 -85.8369906  -87.02805611]
 [-80.         -44.23451777 -66.21052632]
 [-68.76865672 -77.92592593 -80.08108108]]


In [25]:
def calculate_rms(rssi, rssi_ref):
    # Calculate RMS difference for each receiver   
    rms_diff = np.sqrt(((rssi) - rssi_ref)**2)

    return rms_diff

def define_weights(df_rssi, ref_values, iterations):
    # Create a DataFrame to store the results of different weight combinations
    results = pd.DataFrame(columns=['weight_station1', 'weight_station2', 'weight_station3', 'weight_rssi', 'error'])

    # Iterate through weight combinations
    for w_station1 in np.linspace(0, 1, iterations):
        for w_station2 in np.linspace(0, 1, iterations):
            for w_station3 in np.linspace(0, 1, iterations):
                for w_rssi in np.linspace(0, 1, iterations):

                    if w_station1 + w_station2 + w_station3 + w_rssi == 1:
                        errors = []
                        for index, row in df_rssi.iterrows():
                            if row['real station'] == 0:
                                continue  # Skip rows with real station 0

                            # Calculate weighted RMS values for each station using reference values
                            rms_station = np.zeros(3)

                            for station in range(3):
                                rms_station[station] = (w_station1 * calculate_rms(row['rssi1'], ref_values[station, 0]) + 
                                                        w_station2 * calculate_rms(row['rssi2'], ref_values[station, 1]) +
                                                        w_station3 * calculate_rms(row['rssi3'], ref_values[station, 2]))

                            # Determine the station with the minimum weighted RMS value
                            station_min_ref = np.argmin(rms_station) + 1

                            # Calculate the station determined by the smallest RSSI
                            station_min_rssi = np.argmin([row['rssi1'], row['rssi2'], row['rssi3']]) + 1

                            # Calculate the weighted combination of station predictions
                            station = int((w_station1 + w_station2 + w_station3) * station_min_ref + w_rssi * station_min_rssi)

                            # Calculate the error (absolute difference) between predicted station and real station

                            if station == 1 and (row['real station'] == 2 or row['real station'] == 3):
                                error = 3
                            if (station == 2 or station == 3) and row['real station'] == 1:
                                error = 3
                            else:
                                error = 1
                            errors.append(error)

                        # Calculate the mean error for this weight combination
                        mean_error = np.mean(errors)

                        # Store the weight combination and its corresponding error in the results DataFrame
                        results = pd.concat([results, pd.DataFrame({'weight_station1': [w_station1], 'weight_station2': [w_station2],
                                            'weight_station3': [w_station3], 'weight_rssi': [w_rssi],
                                            'error': [mean_error]})], ignore_index=True)
                        
    # Find the weight combination with the minimum error
    best_weights = results.loc[results['error'].idxmin()]
 
    print(best_weights)
    return best_weights


def classify_points(df_rssi, best_weights, ref_values):

    for index, row in df_rssi.iterrows():
        # Calculate weighted RMS values for each station using reference values
        rms_station = np.zeros(3)

        for station in range(3):
            rms_station[station] = (
                best_weights['weight_station1'] * calculate_rms(row['rssi1'], ref_values[station, 0]) +
                best_weights['weight_station2'] * calculate_rms(row['rssi2'], ref_values[station, 1]) +
                best_weights['weight_station3'] * calculate_rms(row['rssi3'], ref_values[station, 2])
            )

        # Determine the station with the minimum weighted RMS value
        station_pred = np.argmin(rms_station) + 1

        df_rssi.loc[index, 'station'] = station_pred


    return df_rssi

def calculate_accuracy(df_rssi):
    accuracy = np.zeros(3)  # Initialize an array to store accuracy values for each station

    for station in range(1, 4):
        total_points = 0
        total_correct = 0

        for index, row in df_rssi.iterrows():
            if row['real station'] == 0:
                continue  # Skip rows with real station 0

            if row['real station'] == station:
                total_points += 1
                if row['station'] == row['real station']:
                    total_correct += 1

        # Calculate accuracy for the current station
        accuracy[station - 1] = total_correct / total_points * 100 if total_points > 0 else 0
    print(accuracy)
    return accuracy


if date == '24_08':
    best_weights = define_weights(df_rssi2, ref_values, iterations=11)
    df_rssi2 = classify_points(df_rssi2,best_weights,ref_values)
    calculate_accuracy(df_rssi2)
else:
    best_weights = {
    'weight_station1': 0.6,
    'weight_station2': 0.0,
    'weight_station3': 0.0,
    'weight_rssi': 0.4,
    'error': 0.5
}
if date == '11_10':
    df_rssi2['real station'] = 1
    df_rssi2 = classify_points(df_rssi2,best_weights,ref_values)
    #print(df_rssi2)
    df_rssi1['real station'] = 2
    df_rssi1 = classify_points(df_rssi1,best_weights,ref_values)
    print(df_rssi1)
#calculate_accuracy(df_rssi2)

#else: 
#    best_weights =
#df_rssi1 = classify_points(df_rssi2,best_weights,ref_values)
#print(df_rssi1)


                    ts rssi1 rssi2 rssi3 station  real station
0  2023-10-11 19:52:58 -60.0 -40.0 -75.0       1             2
1  2023-10-11 20:47:57 -60.0 -40.0 -75.0       1             2


In [37]:
print("Beacon 1")
df_rssi1 = classify_points(df_rssi1,best_weights,ref_values)
#time_in_station_ble(df_rssi1)
display(df_rssi1.loc[:50])
#display(df_rssi1)

print("Beacon 2")
df_rssi2 = classify_points(df_rssi2,best_weights,ref_values)
#time_in_station_ble(df_rssi2)
display(df_rssi2.loc[:50])


Beacon 1


KeyError: 'real state'

In [7]:


def classify_point_simple(df_rssi, ref_values, weight):
    for index, row in df_rssi.iterrows():
        # Calculate RMS values for each station using reference values
        rms_station = np.zeros(3)

        for station in range(3):
            rms_station[station] = (calculate_rms(row['rssi1'], ref_values[station, 0]) + 
                                    calculate_rms(row['rssi2'], ref_values[station, 1]) +
                                    calculate_rms(row['rssi3'], ref_values[station, 2]))

        # Determine the station with the minimum RMS value (0 1 or 2) -> (1 2 or 3)
        station_min_ref = np.argmin(rms_station) + 1

        # Calculate the station determined by the smallest RSSI
        station_min_rssi = np.argmin([row['rssi1'], row['rssi2'], row['rssi3']]) + 1

        # Calculate the weighted station
        station = int(weight * station_min_ref + (1 - weight) * station_min_rssi)

        # Assign the identified station to the dataframe
        df_rssi.loc[index, 'station'] = station

    return df_rssi

def time_in_station_ble(df_rssi):
    # Initialize time variables for each station
    time_in_1 = pd.to_timedelta('0 seconds')
    time_in_2 = pd.to_timedelta('0 seconds')
    time_in_3 = pd.to_timedelta('0 seconds')

    for index, row in df_rssi.iterrows():
        # Calculate time spent in each station based on station assignments
        if row['station'] == 1:
            if index == 0:
                continue
            time_in_1 += df_rssi.loc[index, 'ts'] - df_rssi.loc[index - 1, 'ts']

        if row['station'] == 2:
            if index == 0:
                continue
            time_in_2 += df_rssi.loc[index, 'ts'] - df_rssi.loc[index - 1, 'ts']

        if row['station'] == 3:
            if index == 0:
                continue
            time_in_3 += df_rssi.loc[index, 'ts'] - df_rssi.loc[index - 1, 'ts']

    # Calculate total time and percentages for each station
    total_time = time_in_1 + time_in_2 + time_in_3
    perc1 = time_in_1 / total_time * 100
    perc1 = round(perc1, 2)
    perc2 = time_in_2 / total_time * 100
    perc2 = round(perc2, 2)
    perc3 = time_in_3 / total_time * 100
    perc3 = round(perc3, 2)
    print(time_in_1,time_in_2,time_in_3)
    print(time_in_1+time_in_2+time_in_3)
    print(f'station1: {perc1}%,    station2: {perc2}%,    station3: {perc3}%')
    return


print("Beacon 1")
df_rssi1 = classify_point(df_rssi1, ref_values,weight=0.5)
time_in_station_ble(df_rssi1)
display(df_rssi1.loc[:50])
#display(df_rssi1)

print("Beacon 2")
df_rssi2 = classify_point(df_rssi2, ref_values,weight=0.5)
time_in_station_ble(df_rssi2)
display(df_rssi2.loc[:50])




Beacon 1
0 days 00:29:14 0 days 00:09:07 0 days 00:00:00
0 days 00:38:21
station1: 76.23%,    station2: 23.77%,    station3: 0.0%


,ts,rssi1,rssi2,rssi3,station
0,2023-08-24 17:30:37,-47.5,-100,-54.0,1
1,2023-08-24 17:31:49,-46.0,-100,-70.0,1
2,2023-08-24 17:32:25,-48.333333,-100,-70.0,1
3,2023-08-24 17:32:31,-46.333333,-100,-70.0,1
4,2023-08-24 17:32:49,-49.0,-100,-68.0,1
5,2023-08-24 17:33:25,-47.0,-100,-62.5,1
6,2023-08-24 17:33:37,-53.666667,-100,-69.5,1
7,2023-08-24 17:34:55,-53.333333,-100,-67.0,1
8,2023-08-24 17:35:01,-57.666667,-100,-67.0,1
9,2023-08-24 17:35:37,-58.0,-100,-66.5,1


Beacon 2
0 days 00:27:20 0 days 00:11:49 0 days 00:00:00
0 days 00:39:09
station1: 69.82%,    station2: 30.18%,    station3: 0.0%


,ts,rssi1,rssi2,rssi3,station
0,2023-08-24 17:30:07,-64.5,-100,-58.0,1
1,2023-08-24 17:30:13,-65.333333,-100,-58.0,1
2,2023-08-24 17:30:31,-62.666667,-100,-58.0,1
3,2023-08-24 17:30:43,-65.666667,-100,-58.0,1
4,2023-08-24 17:31:19,-67.666667,-100,-58.0,1
5,2023-08-24 17:32:31,-69.666667,-100,-75.0,1
6,2023-08-24 17:33:19,-67.0,-100,-35.0,1
7,2023-08-24 17:33:43,-65.0,-100,-70.0,1
8,2023-08-24 17:33:49,-63.0,-100,-70.0,1
9,2023-08-24 17:34:25,-66.0,-100,-70.0,1


In [8]:
# Resampling with forward fill
def resample_df(df, freq):
    # Set the 'Timestamp' column as the DataFrame index
    df.set_index('ts', inplace=True)

    # Resample the DataFrame with a desired frequency (e.g., 10 minutes) and forward fill
    resampled_df = df.resample(freq).ffill()

    # Reset the index to have the timestamp as a column again
    resampled_df = resampled_df.reset_index()

    return resampled_df

#display(df1.loc[:50])
df_rssi1_ = resample_df(df_rssi1,freq = '5S')
df_rssi2_ = resample_df(df_rssi2,freq = '5S')


In [9]:
    
def plot_ble_state(df_rssi,beacon):
 
    x = np.array(df_rssi['ts'])
    y = np.array(df_rssi['station'])
    fig1 = go.Figure()
    fig1.add_trace(go.Scatter(x=x, y=y, name="hv",
                    line_shape='hv'))
    fig1.show()


plot = 1

if plot:
    plot_ble_state(df_rssi1_,beacon = 1)
    plot_ble_state(df_rssi2_, beacon = 2)



   

In [ ]:
def heat_map(df):

    labels_nr = [0,1,2,3,4,5]

    colors = ['#440154', '#3E4989', '#26828E' , '#35B779', '#6ECE58', '#FDE725']

    dcolorsc = discrete_colorscale(labels_nr, colors)

    step = max(labels_nr)/(len(labels_nr))
    nvals = [round(i*step,2) for i in range(len(labels_nr)+1)]
    tickvals = [(float(nvals[k])+float(nvals[k+1]))/2 for k in range(len(nvals)-1)] #position with respect to labels_nr where ticktext is displayed
    ticktext = [f'{labels_nr[k]}' for k in range(0, len(labels_nr))]
    #dcolorsc = 'viridis'
    #dcolorsc = [[0, 'red'], [1, 'green'], [2, 'yellow'], [3, 'blue'], [4, 'black'], [5, 'brown']]

    fig = go.Figure(data=go.Heatmap(
            z=df['label_aux'],
            x=df['ts_aux'],
            y=df['label_aux'],
            colorscale=dcolorsc,
            colorbar=dict(
                title="Labels",
                thickness=15,
                tickvals=tickvals,
                ticktext=ticktext,
            )
    ))

    # add title, change background color, change grid, remove colorbar
    fig.update_layout(
        title='CNC Working States',
        plot_bgcolor='white',
        xaxis_showgrid=False, yaxis_showgrid=False,xaxis_zeroline=False, yaxis_zeroline=False,
        coloraxis_showscale=False
    )


    # add some time reference lines to the xaxis
    tickvals = pd.date_range(start=df['ts_aux'].min(), end=df['ts_aux'].max(), freq='12H')
    ticktext = [val.strftime('%Y-%m-%d %H:%M:%S') for val in tickvals]
    tickvals_start = [val - pd.Timedelta(hours=3) for val in tickvals]
    ticktext_start = [val.strftime('|<br>%Y-%m-%d<br>%H:%M:%S') for val in tickvals_start]
    fig.update_xaxes(type="date", tickmode='array', tickvals=tickvals_start, ticktext=ticktext_start)

    labels = ['Idle','Tool measurement','Tool change','Machining','Probing','Pallet change']

    fig.update_yaxes(ticktext=labels, tickvals=[0,1,2,3,4,5])
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey', tickson='labels')

    fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)

    fig.show()

In [10]:
def strip_plot(df, beacon):
    df['ts'] = df['ts'].astype(str)  

    y_scale = alt.Scale(domain=[1, 2, 3])  # Set the y-axis range from 1 to 3

# Define color encoding based on the station value
    color = alt.Color('station:N', scale=alt.Scale(
        domain=[1, 2, 3], range=['cornflowerblue', 'darkorange', 'violet']
    ))

    chart = alt.Chart(df).mark_tick().encode(
        x=alt.X('ts:T', title='Timestamp'),
        y=alt.Y('station:O', title='Station', scale=y_scale),
        color=color  # Apply the color encoding
    ).properties(
        title=f'Classified station for beacon {beacon}',
        width=300  # Set the width to 600 pixels (adjust as needed)
        #height=150  # Set the height to 400 pixels (adjust as needed)
    )

    return chart

    

chart1 = strip_plot(df_rssi1_,beacon=1)
chart2 = strip_plot(df_rssi2_,beacon=2)

chart1 | chart2

alt.HConcatChart(...)